In [56]:
# some example data
import pandas as pd
from typing import Union
from statsmodels.tsa.vector_ar.var_model import VARResultsWrapper
from statsmodels.tsa.api import VAR

from statsmodels.tsa.base.datetools import dates_from_str
import numpy as np

In [157]:
import yfinance as yf

# 2023.TW 燁輝
# 2002.TW 中鋼
# 2006.TW 中和鋼鐵
# 2007.TW 燁興
date = '2016-01-01'
stock_no = '2002.TW'

stock = yf.Ticker(stock_no)
data = stock.history(start=date)

data#.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04 00:00:00+08:00,13.072967,13.072967,12.709829,12.709829,24549000,0.00,0.0
2016-01-05 00:00:00+08:00,12.709828,12.964025,12.564573,12.600887,27544000,0.00,0.0
2016-01-06 00:00:00+08:00,12.637202,12.709830,12.564574,12.564574,18716000,0.00,0.0
2016-01-07 00:00:00+08:00,12.564574,12.709830,12.383005,12.673516,24452000,0.00,0.0
2016-01-08 00:00:00+08:00,12.564573,12.746142,12.455632,12.637201,20183000,0.00,0.0
...,...,...,...,...,...,...,...
2024-07-18 00:00:00+08:00,23.094698,23.242425,22.946970,23.143940,28548483,0.00,0.0
2024-07-19 00:00:00+08:00,23.094697,23.143939,22.750000,22.996212,22809555,0.00,0.0
2024-07-22 00:00:00+08:00,22.897727,22.946969,22.651515,22.750000,26506583,0.00,0.0


In [7]:
#data = pd.read_csv(dir+"National Stock Exchange/infy_stock.csv")

In [58]:
# """
# 要把 dataframe 的格式清理成以下格式
# dates_from_str 要放入 index
# 剩下的資料只能保留 feature
# """
# # statsmodels的範例資料
# import statsmodels.api as sm
# mdata = sm.datasets.macrodata.load_pandas().data
# mdata
# dates = mdata[['year', 'quarter']].astype(int).astype(str)
# quarterly = dates["year"] + "Q" + dates["quarter"]
# quarterly = dates_from_str(quarterly)
# mdata.index = pd.DatetimeIndex(quarterly)
# data = mdata.drop(columns=["year","quarter"])
# data

# # KAGGLE 範例資料 1 
# dir = "C:/Users/foresight_User/Desktop/公司文件/9.產品/AIUPS/"
# data = pd.read_csv(dir+"National Stock Exchange/infy_stock.csv")
# data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
# data = data.drop(columns=["Date","Symbol","Series"])
# data

# # KAGGLE 範例資料 2
# data = pd.read_csv(dir+"National Stock Exchange/nifty_it_index.csv")
# data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
# data = data.drop(columns=["Date"])
# data

# # KAGGLE 範例資料 3
# data = pd.read_csv(dir+"National Stock Exchange/tcs_stock.csv")
# data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
# data = data.drop(columns=["Date","Symbol","Series"])
# data

In [59]:
def time_series_format_preprocessing(df:pd.DataFrame, datetime_col:str):
    #
    # YS(年初), MS(月初), W(周), D(日), H(小時), T(分鐘), S(秒),
    #
    # df = df.copy()
    # df[datetime_col] = pd.to_datetime(df[datetime_col], format='%Y-%m-%d %H:%M:%S')
    # df = df.set_index(datetime_col)
    # df = df.asfreq(interval)
    # return df if set_index_flag else df.reset_index(drop=False)
    df.index =pd.DatetimeIndex(dates_from_str(data[datetime_col]))
    df = df.drop(columns=[datetime_col])
    """
    Imputes missing values in a time series DataFrame using the specified function.

    Args:
        df (pd.DataFrame): The input DataFrame containing the time series data.
        func (str): The imputation function to be applied. Supported options are:
                    - 'forward_fill': Forward fill missing values.
                    - 'backward_fill': Backward fill missing values.
                    - 'moving_average': Impute missing values using moving average.
                    - 'interpolation': Perform linear interpolation to fill missing values.
        col_name (str): The name of the column to impute missing values.

    Returns:
        pd.DataFrame: The DataFrame with missing values imputed based on the specified function.
    """
    return df


def time_series_impute_missing_value(df, func, col_name):
    """
    Imputes missing values in a time series DataFrame using the specified function.

    Args:
        df (pd.DataFrame): The input DataFrame containing the time series data.
        func (str): The imputation function to be applied. Supported options are:
                    - 'forward_fill': Forward fill missing values.
                    - 'backward_fill': Backward fill missing values.
                    - 'moving_average': Impute missing values using moving average.
                    - 'interpolation': Perform linear interpolation to fill missing values.
        col_name (str): The name of the column to impute missing values.

    Returns:
        pd.DataFrame: The DataFrame with missing values imputed based on the specified function.
    """
    df = df.copy()
    if func == 'forward_fill':
        df[col_name].ffill(inplace=True)
    elif func == 'backward_fill':
        df[col_name].bfill(inplace=True)
    elif func == 'moving_average':
        df[col_name].fillna(df[col_name].rolling(window=3, min_periods=1).mean(), inplace=True)
    elif func == 'interpolation':
        df[col_name].interpolate(inplace=True)
    else:
        raise ValueError(f"Unsupported imputation function: {func}")

    return df

In [188]:
def cal_maxLag(data:pd.DataFrame):
    n_totobs = len(data)
    ntrend = 1 #len(trend) if trend.startswith("c") else 0
    neqs = data.shape[1]
    max_estimable = (n_totobs - neqs - ntrend) // (1 + neqs)
    if max_estimable > 1:
        return max_estimable
    else:
        return 1

In [61]:

def vectorAutoregression(data:pd.DataFrame,maxlags:Union[int,str]="auto",ic:str=None):
    model = VAR(data)
    # ==== 這邊不要動 =====

    """ 
    這是套件設定的
    trend : str {"n", "c", "ct", "ctt"}
        * "n" - no deterministic terms
        * "c" - constant term
        * "ct" - constant and linear term
        * "ctt" - constant, linear, and quadratic term

    maxlags 不可以超過 max_estimable 的值
    maxlags 為模型擬合最大數值
    statemodel有設定條件，已經寫在下述的程式
    使用者要調整低於 maxlags
    """
    max_estimable = cal_maxLag(data)
    # ==== 這邊不要動 =====

    print(" maxlags 要小於等於: ", max_estimable)

    if maxlags == "auto":
        maxlags = max_estimable
    if maxlags > max_estimable:
        raise Exception(" maxlags 要小於等於: ", max_estimable)


    """
    ic 為評估模型的好壞
    ic = {'aic', 'fpe', 'hqic', 'bic', None}
    Information criterion to use for VAR order selection.
    aic : Akaike
    fpe : Final prediction error
    hqic : Hannan-Quinn
    bic : Bayesian a.k.a. Schwarz
    """

    results = model.fit(maxlags=maxlags, ic=ic)
    print(f"在最大 lag 數目為 {max_estimable} 的情況下，VAR 找出的最佳 lag 為: ",results.k_ar)
    return results

In [152]:
def vectorAutoregressionRelationship(results:VARResultsWrapper,target:str,pvalue_threshold:float=0.05):
    # target  客人關心的 Y 是甚麼，Y 會包含在 results 中

    coef_df = results.params[target]
    pvalues_df = results.pvalues[target]

    # 合并系数和p值
    summary = pd.concat([coef_df, pvalues_df], axis=1)
    summary.columns = ['coef', 'pvalue']
    summary = summary.drop(index="const").reset_index()
    summary_index = summary["index"].str.split(".", expand=True).rename(columns={0:"time_lag",1:"feature"})
    if summary.empty:
        return {"info":"there is no results found from VAR"}
    else:
        summary_index["time_lag"] = summary_index["time_lag"].str.replace("L","").astype(int)
        summary = pd.concat([summary_index,summary],axis=1).drop(columns="index")
        summary = summary[summary["pvalue"]<pvalue_threshold].reset_index(drop=True)
        return summary

In [178]:



# 定義聚合函數和相應的列名
operator_list = ['first', 'last', 'mean','prod','median','max','min']#'std','max','count','sum',"min","prod","median","var","sem","skew","cumsum","cumprod"]


aggregation_functions = {}
for i in data.columns:
    aggregation_functions[i]=operator_list

# 按週重採樣並應用聚合
weekly_data = data.resample('W').agg(aggregation_functions)

# 自定義列名

monthly_data = data.resample('M').agg(aggregation_functions)
yearly_data = data.resample('Y').agg(aggregation_functions)

weekly_data.columns = ['_'.join(col).strip() for col in weekly_data.columns.values]
monthly_data.columns = ['_'.join(col).strip() for col in monthly_data.columns.values]
yearly_data.columns = ['_'.join(col).strip() for col in yearly_data.columns.values]


In [183]:

def remove_collinearity(data): #condition
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    #data = data.fillna(method='ffill').fillna(method='bfill')
    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.ffill().bfill() 
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]


    # 去除VIF為NAN或INF
    vif_data = vif_data.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    data = data[vif_data["feature"].to_list()]

    # 移除高相關的 feature 
    # 保留低相關的 feature
    coefficient_threshold = 0.3
    correlation_matrix = data.corr()
    low_corr_vars = np.where(np.abs(correlation_matrix) < coefficient_threshold)
    #print(correlation_matrix)
    low_corr_vars = [(correlation_matrix.index[x], correlation_matrix.columns[y]) for x, y in zip(*low_corr_vars) if x != y and x > y]

    # Remove one of each highly correlated pair
    # if condition == "intersection":
    #     sets =  [set(pair) for pair in low_corr_vars]
    #     intersection = set.intersection(*sets)
    #     remaining_list = list(intersection)
    # if condition == "union":
    unique_elements = set(element for pair in low_corr_vars for element in pair)
    remaining_list = list(unique_elements)
    
    data = data[remaining_list]
    
    # impute nan
    #data = data.fillna(method='ffill').fillna(method='bfill')
    return data

In [190]:
"""
讓使用者決定好奇的target是甚麼，套件會自動找出所有跟target相關的不同time lag的時間變相
maxlags 會影響最終最佳推薦的結果，也得慎選，但設定上不可以超過 cal_maxLag 輸出的值
"""
# #data = pd.read_csv(dir+"National Stock Exchange/tcs_stock.csv").drop(columns=["Symbol","Series"])
# data = time_series_format_preprocessing(data,"Date")
# data = time_series_impute_missing_value(data, 'interpolation', "Volume")

# 如果有缺失值，可以选择填充或删除


# 
# 
# 
for d in [data,weekly_data,monthly_data,yearly_data]:
    clean_data = remove_collinearity(data=d)
    maxlags = cal_maxLag(clean_data) # USER 不能指定超過這個的LAG，LAG值也要大於0
    print(maxlags)
    for i in clean_data.columns:
        target = i #"Open_first"#"Trades"
        print(target)
        results = vectorAutoregression(clean_data,maxlags=maxlags,ic=None)
        pvalue_threshold = 0.05
        VAR_relationship = vectorAutoregressionRelationship(results=results,target=target,pvalue_threshold=pvalue_threshold)

c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\regression\linear_model.py:1785: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


296
Dividends
 maxlags 要小於等於:  296
在最大 lag 數目為 296 的情況下，VAR 找出的最佳 lag 為:  296
Open
 maxlags 要小於等於:  296


c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


在最大 lag 數目為 296 的情況下，VAR 找出的最佳 lag 為:  296
Close
 maxlags 要小於等於:  296


c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


...

c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.cov_params()))
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


1
Volume_prod
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
High_first
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Low_min
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Open_last
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Open_first
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Open_max
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Close_median
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Volume_min
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Dividends_max
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Open_prod
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Volume_median
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Close_mean
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Low_mean
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Low_first
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Open_median
 maxlags 要小於等於:  1
在最

c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\numpy\linalg\linalg.py:1900: RuntimeWarning: overflow encountered in multiply
  tol = S.max(axis=-1, keepdims=True) * max(A.shape[-2:]) * finfo(S.dtype).eps
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\regression\linear_model.py:1734: RuntimeWarning: overflow encountered in square
  return np.sum(weights * (model.endog - mean)**2)
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.cov_params()))
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.c

Open_max
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Close_median
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Open_first
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Volume_min
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Dividends_max
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Volume_median
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Close_mean
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Low_mean
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Low_first
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Open_median
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
High_last
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
High_mean
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Open_min
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
High_median
 maxlags 要小於等於:  1
在最大 lag 數目為 1 的情況下，VAR 找出的最佳 lag 為:  1
Volume_first
 maxlags 要小於等於:  1
在

c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.cov_params()))
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.cov_params()))
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.cov_params()))
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.cov_params()))
c:\Users\User\.conda\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.cov_params()))
c:\Us

In [191]:
VAR_relationship

,time_lag,feature,coef,pvalue
0,1,High_first,-2.122830e-01,0.000000e+00
1,1,Low_min,2.212313e-01,0.000000e+00
2,1,Open_max,-1.976034e-01,0.000000e+00
3,1,Open_first,-1.792636e-01,0.000000e+00
4,1,Volume_min,2.415786e-07,0.000000e+00
5,1,Volume_median,6.873067e-07,0.000000e+00
6,1,Low_mean,2.197454e-02,1.011477e-97
7,1,Open_median,-1.491220e-02,5.375936e-207
8,1,High_last,1.817686e-01,0.000000e+00
9,1,Volume_first,1.873044e-07,0.000000e+00


In [148]:
coef_df = results.params[target]
pvalues_df = results.pvalues[target]

# 合并系数和p值
summary = pd.concat([coef_df, pvalues_df], axis=1)
summary.columns = ['coef', 'pvalue']
summary = summary.drop(index="const").reset_index()

In [155]:
VAR_relationship

{'info': 'there is no results found from VAR'}

In [147]:
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 26, Jul, 2024
Time:                     16:48:27
--------------------------------------------------------------------
No. of Equations:         7.00000    BIC:                    39.6641
Nobs:                     9.00000    HQIC:                   39.1796
Log likelihood:          -260.191    FPE:                1.45234e+17
AIC:                      39.5107    Det(Omega_mle):     6.94651e+16
--------------------------------------------------------------------
Results for equation Dividends
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.177975         0.056955            3.125           0.002

Results for equation Open
           coefficient       std. error           t-stat            prob
-------------------------------------------------------------

In [29]:
data


,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
2015-01-01,2558.25,2567.00,2567.00,2541.00,2550.00,2545.55,2548.51,183415,4.674345e+13,8002,52870,0.2883
2015-01-02,2545.55,2551.00,2590.95,2550.60,2588.40,2579.45,2568.19,462870,1.188740e+14,27585,309350,0.6683
2015-01-05,2579.45,2581.00,2599.90,2524.65,2538.10,2540.25,2563.94,877121,2.248886e+14,43234,456728,0.5207
2015-01-06,2540.25,2529.10,2529.10,2440.00,2450.05,2446.60,2466.90,1211892,2.989615e+14,84503,714306,0.5894
2015-01-07,2446.60,2470.00,2479.15,2407.45,2426.90,2417.70,2433.96,1318166,3.208362e+14,101741,886368,0.6724
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-24,2425.80,2421.00,2438.20,2404.00,2435.00,2434.25,2424.03,421580,1.021924e+14,21932,188670,0.4475
2015-12-28,2434.25,2428.00,2466.40,2420.20,2456.95,2462.70,2455.00,1852099,4.546896e+14,43390,1060100,0.5724
2015-12-29,2462.70,2458.35,2465.30,2445.75,2449.35,2455.80,2452.26,854262,2.094875e+14,26975,433986,0.5080
2015-12-30,2455.80,2453.05,2459.00,2412.30,2421.70,2418.30,2442.06,802881,1.960682e+14,49464,632180,0.7874


In [28]:
"""
生成VAR關係公式
"""
"""
可有可無，看需求
"""
formula_parts = []
for index, row in VAR_relationship.iterrows():
    coef = row['coef']
    if coef < 0:
        term = f"- {-coef:.3f} * {row['feature']}(t-{row['time_lag']})"
    else:
        term = f"{coef:.3f} * {row['feature']}(t-{row['time_lag']})"
    formula_parts.append(term)

formula = " + ".join(formula_parts).replace("+ -", "- ")
target_formula = f"f{target}(t) = {formula}"

# 打印公式
print(target_formula)

fTrades(t) = 0.553 * Trades(t-1)
